# Data and Variables

We need to represent and access info for all passengers: gender(weight), which group they belong to, if they have to transfert. We created a class Passenger to represent this data.

The main variable here is the allocation : binary variable that says for each passenger where he is located : 

allocation = m.addVars(len(passengers), width, height, vtype=GRB.BINARY, name="Allocation")

There are $$len(passengers)*width*heigh$$ variables. For instance, if Passenger 1 is at the place : width=2, height=3, then allocation[1,2,3]=1. Else, it equals 0.

There are also the satisfaction variables: depends on the number of neighbors of the same group you have [to be completed]

# Constraints

### At most 1 person per seat constraint: in the plane, there can't be more than 1 person in each seat.

for p in passengers,
$$\sum_{passenger = 1}^{p}allocation[p.passenger_id,i,j]  <= 1 $$for i in range(width) for j in range(height)

### Everyone has 1 and only 1 seat constraint: each passenger has to be seated in only one place.
$$\sum_{i,j = 1}^{w,h}allocation[p.passenger_id,i,j] ==1 $$

for p in passengers

With w the width (X), h the height (Y)

### Barycenter constraint : separate the X and Y axis. Then separate each axis into 2 constraints: 1 lower constraint, 1 upper constraint, to fit into the defined barycenter area. 

$$\sum_{passenger = 1}^{p}[p.weight * allocation[p.passenger_id,i,j]*X[i]  /total_{weight} <= 1$$ for i in range(width) for j in range(height)

$$\sum_{passenger = 1}^{p}[p.weight * allocation[p.passenger_id,i,j]*X[i]  /total_{weight} >= -1$$ for i in range(width) for j in range(height)

$$\sum_{passenger = 1}^{p}[p.weight * allocation[p.passenger_id,i,j]*Y[i]  /total_{weight} <= 2$$ for i in range(width) for j in range(height)

$$\sum_{passenger = 1}^{p}[p.weight * allocation[p.passenger_id,i,j]*Y[i]  /total_{weight} >= -2$$ for i in range(width) for j in range(height)

##### With X[i] being the distance from the passenger to the barycenter on the X axis and Y[i] being the distance from the passenger to the barycenter on the Y axis


### Constraint of transfert passenger : if a passenger has to transfert, he needs to be in the first third of the plane (Y axis). No constraint on the X axis. If there is no transfert, the passenger can be anywhere.


 For p in passengers, if $$ p.transfert=1$$  $$allocation[p.passenger_{id}][y] \leq height/3 \;\;\;\;  \;\;\;$$


# Constraints relative to the objective function

The constraints were nonlinear at the beginning, it worked for small groups of passengers but we then had to linearize them in order to make it work for all cases.

# Objective function

The objective here is to maximize the satisfaction of the passengers. The satisfaction depends on the number of neighbors of the same group each passenger has, and is calculated like this:
- For groups of 3 passengers or less:
    1 point for each neighbor of the same group in the same row, not separated by the aisle or a passenger of another group
- For groups of 4 passengers or more:
    - 1 point for each neighbor of the same group in the same row, not separated by a passenger of another group
    - 0.5 points for each neighbor of the same group behind or in front, only and only if you already have 1 neighbor of your group on one of your sides
    
In general, we will try to add points if the passengers of the same group are close together even if they're not exact neighbors.
We created a satisfaction coefficient for the groups of less than 3 passengers and another one for the groups with more than 3 passengers. 

The objective function sums up all the satisfaction coefficients for all passengers and has to be maximized.

$$\mbox{Maximize} \;\;\; \sum_{passenger = 1}^{p} S1_^p + S2_p$$
